In [7]:
# score coherence
from sentence_transformers import SentenceTransformer, util

GLOSS = {
    "business": ["đặt phòng khám", "tái khám", "hẹn lịch khám", "đo kính mắt", "mua bán"],
    "expertise": ["thị giác", "nhược thị", "tật khúc xạ", "bệnh về mắt", "khám mắt cho trẻ em", "bác sĩ mắt"],
}

def score_coh(sentence, model):
    s_embd = model.encode(sentence)
    domains = list(GLOSS.keys())
    
    score = 0
    
    for k in domains:
        for term in GLOSS[k]: 
            w_embd = model.encode(term)
            score += util.cos_sim(w_embd, s_embd)
    
    return score    

In [12]:
model_semantic = SentenceTransformer("./vietnamese-sbert/")
score_coh("Em có con nhỏ đang bị cận muốn cho đi khám bác ạ được không ạ", model_semantic)

tensor([[3.0575]])

In [13]:
# score interest
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

def score_interest(sentence, model, tokenizer):
    input_ids = torch.tensor([tokenizer.encode(sentence)])
    with torch.no_grad():
        out = model(input_ids)
        # print(out.logits.softmax(dim=-1).tolist())
    return out.logits.softmax(dim=-1).tolist() 

# Just like PhoBERT: INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!


In [14]:
model = RobertaForSequenceClassification.from_pretrained("./phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("./phobert-base-vietnamese-sentiment", use_fast=False)

score_interest("được đấy", model, tokenizer)

[[0.007860301062464714, 0.8275406956672668, 0.16459901630878448]]